In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import chardet
from io import StringIO

In [2]:
SCORE_DIR = "data/scores"

box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]
box_score = box_scores[0]

In [3]:
#cleaning the table and removing some columns/rows
def parse_html(box_score):
    with open(box_score, 'rb') as f:
        result = chardet.detect(f.read())
    detected_charset = result['encoding']
    with open(box_score, encoding=detected_charset) as f:
        html = f.read()
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [4]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team", "total"]]
    return line_score

In [5]:
def read_stats(soup, team, stat):
    df = pd.read_html(StringIO(str(soup)), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [6]:
def read_season_info(soup):
    #selecting first in list
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [7]:
base_cols = None
games = []
for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        #get basic and advanced states
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        #gathering the total of both basic and advance stats and making it one row
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
        
        #gathering the max and making it a row
        players_to_track = 8
        summary = totals
        for i in range(players_to_track):
            player_data = pd.concat([basic.iloc[i], advanced.iloc[i]])
            player_data.index = player_data.index.str.lower() + f"_{i+1}"
            # concat both max and totals together
            summary = pd.concat([summary, player_data])

        #cleaning cols that we want and removing duplicates
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]

        summaries.append(summary)
    #turning the data frame
    summary = pd.concat(summaries, axis=1).T
    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [1, 0]

    #making game in perspective of opposition
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    #combining both such that a row contains both the team and opponent scores
    full_game = pd.concat([game, game_opp], axis=1)

    #adding season to help provide information to training model
    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]

    mp_cols = full_game.filter(like='mp').columns
    full_game.drop(columns=mp_cols, inplace=True)

    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 10206
200 / 10206
300 / 10206
400 / 10206
500 / 10206
600 / 10206
700 / 10206
800 / 10206
900 / 10206
1000 / 10206
1100 / 10206
1200 / 10206
1300 / 10206
1400 / 10206
1500 / 10206
1600 / 10206
1700 / 10206
1800 / 10206
1900 / 10206
2000 / 10206
2100 / 10206
2200 / 10206
2300 / 10206
2400 / 10206
2500 / 10206
2600 / 10206
2700 / 10206
2800 / 10206
2900 / 10206
3000 / 10206
3100 / 10206
3200 / 10206
3300 / 10206
3400 / 10206
3500 / 10206
3600 / 10206
3700 / 10206
3800 / 10206
3900 / 10206
4000 / 10206
4100 / 10206
4200 / 10206
4300 / 10206
4400 / 10206
4500 / 10206
4600 / 10206
4700 / 10206
4800 / 10206
4900 / 10206
5000 / 10206
5100 / 10206
5200 / 10206
5300 / 10206
5400 / 10206
5500 / 10206
5600 / 10206
5700 / 10206
5800 / 10206
5900 / 10206
6000 / 10206
6100 / 10206
6200 / 10206
6300 / 10206
6400 / 10206
6500 / 10206
6600 / 10206
6700 / 10206
6800 / 10206
6900 / 10206
7000 / 10206
7100 / 10206
7200 / 10206
7300 / 10206
7400 / 10206
7500 / 10206
7600 / 10206
7700 / 10206
7800 / 1

In [8]:
games_df = pd.concat(games, ignore_index=True)

In [9]:
games_df.to_csv("nba_games_with_player_data.csv")